<a href="https://colab.research.google.com/github/LuFernandez/twin-peaks-generator/blob/master/script_generator_moe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [254]:
import keras
import numpy as np
from collections import Counter
from keras.models import Model
from keras.layers import Embedding, Dense, LSTM, Dropout, Input
from sklearn.model_selection import train_test_split

In [255]:
import tensorflow as tf
print('using gpu?' + tf.test.gpu_device_name())

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

using gpu?/device:GPU:0


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2372546937863392575, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14674281152
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 1760917590616257520
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [256]:
# loading the data into file
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My Drive/twin-peaks-generator
#! git clone https://github.com/LuFernandez/twin-peaks-generator
!wget "https://raw.githubusercontent.com/LuFernandez/twin-peaks-generator/master/data/twin_peaks_scripts.txt"
!wget "https://raw.githubusercontent.com/LuFernandez/twin-peaks-generator/master/data/moes_tavern_lines.txt"


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[Errno 2] No such file or directory: 'gdrive/My Drive/twin-peaks-generator'
/content/gdrive/My Drive/twin-peaks-generator
--2021-05-16 19:20:12--  https://raw.githubusercontent.com/LuFernandez/twin-peaks-generator/master/data/twin_peaks_scripts.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1334668 (1.3M) [text/plain]
Saving to: ‘twin_peaks_scripts.txt.12’

twin_peaks_scripts. 100%[===================>]   1.27M  --.-KB/s    in 0.08s   

2021-05-16 19:20:12 (16.3 MB/s) - ‘twin_peaks_scripts.txt.12’ saved [1334668/1334668]

--2021-05-16 19:20:12--  https://raw.githubusercontent.com/LuFernandez/twin-peaks-generator/master/data/

In [257]:
data_dir = 'twin_peaks_scripts.txt'
#data_dir = 'moes_tavern_lines.txt'

with open(data_dir) as f:
    data = f.read()
    
data = data[81:].lower()

In [258]:
data[:447]

' bright eye and beak of the bird, backlit by the first\nlight of the rising sun.\next. interstate highway "21" - dawn\na long, straight, empty stretch of road leads up between two\nmountains, whitetail and blue pine, the twin peaks.\next. twin peaks town square - dawn\nthe picturesque center of the town of twin peaks; a small,\nneatly manicured park with a pristine white gazebo its\ncenterpiece. fronting the park is the \'30\'s-style streamlined,\nconcre'

In [259]:
data[:240].split('\n')

[' bright eye and beak of the bird, backlit by the first',
 'light of the rising sun.',
 'ext. interstate highway "21" - dawn',
 'a long, straight, empty stretch of road leads up between two',
 'mountains, whitetail and blue pine, the twin peaks.',
 'ext. twin p']

In [260]:
data = data[:len(data)//2]

In [261]:
# seperate the punctuations from the words

punch = ['.', '[', ']', '(', ')', ';', ':', "'", '/', '"', ',', '?', '*', '!', '-', '$', '%', '&', '\n']

for i in punch:    
    data = data.replace(i, ' ' + i + ' ')
    
data = data.replace('\n', '<NEWLINE>')    

In [262]:
data[:400]

' bright eye and beak of the bird ,  backlit by the first <NEWLINE> light of the rising sun .  <NEWLINE> ext .  interstate highway  " 21 "   -  dawn <NEWLINE> a long ,  straight ,  empty stretch of road leads up between two <NEWLINE> mountains ,  whitetail and blue pine ,  the twin peaks .  <NEWLINE> ext .  twin peaks town square  -  dawn <NEWLINE> the picturesque center of the town of twin peaks ;'

In [263]:
def get_vocab(text):
    
    vocab_to_int = dict()
    int_to_vocab = dict()
    
    vocab = Counter()
    for word in text.split():
        vocab[word] += 1
        
    index = 0    
    for word in vocab:
        vocab_to_int[word] = index
        int_to_vocab[index] = word
        index += 1
        
    return vocab, vocab_to_int, int_to_vocab

In [264]:
vocab, vocab_to_int, int_to_vocab = get_vocab(data)

In [265]:
print("vocab size:", len(vocab))

vocab size: 8740


In [266]:
# converting text into int

text_int = []

for word in data.split():
    text_int.append(vocab_to_int[word])
    
text_int = np.array(text_int)    

In [267]:
vocab_size = len(vocab)
seq_len = 30
embedding = 300
lstm_size = 128

In [268]:
def get_training_data(data, seq_len):
    
    x_train = []
    y_train = []
    
    for i in range(0, len(data)-seq_len):
        
        x = data[i:i+seq_len]
        y = data[i+1:i+seq_len+1]
        
        x_train.append(np.array(x))
        y_train.append(np.array(y))
        
    return x_train, y_train

In [269]:
x, y = get_training_data(text_int, seq_len)

x = np.array(x)
y = np.array(y)
y = y.reshape(y.shape[0], y.shape[1], 1)

In [270]:
x.shape

(183849, 30)

In [271]:
inp = Input((None,))

embed = Embedding(input_dim=vocab_size, output_dim=embedding)
lstm1 = LSTM(lstm_size, return_sequences=True, return_state=True)
lstm2 = LSTM(lstm_size, return_sequences=True, return_state=True)
lstm3 = LSTM(lstm_size, return_sequences=True, return_state=True)
dense = Dense(vocab_size)

net = embed(inp)
net, h1, c1 = lstm1(net)
net, h2, c2 = lstm2(net)
net, h3, c3 = lstm3(net)
out = dense(net)

model = Model(inp, out)

init_states = [Input((lstm_size,)) for i in range(6)]

inference = embed(inp)
inference, h1, c1 = lstm1(inference, initial_state=init_states[:2])
inference, h2, c2 = lstm2(inference, initial_state=init_states[2:4])
inference, h3, c3 = lstm3(inference, initial_state=init_states[4:6])
inf_out = dense(inference)

states = [h1, c1, h2, c2, h3, c3]
inf_model = Model([inp]+init_states, [inf_out]+states)

In [272]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [273]:
model.optimizer.lr = 0.01
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [274]:
#me aseguro de estar usando GPU
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8164598888158350575, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14674281152
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 10475112380310324074
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [275]:
print(model.fit(x, y, batch_size=128, epochs=20, shuffle=True))
model.save('model.model')

Epoch 1/20
1437/1437 [==============================] - 86s 57ms/step - loss: 7.6540 - accuracy: 0.1436
Epoch 2/20
1437/1437 [==============================] - 82s 57ms/step - loss: 6.6939 - accuracy: 0.2215
Epoch 3/20
1437/1437 [==============================] - 82s 57ms/step - loss: 6.6381 - accuracy: 0.2487
Epoch 4/20
1437/1437 [==============================] - 82s 57ms/step - loss: 6.5749 - accuracy: 0.2672
Epoch 5/20
1437/1437 [==============================] - 82s 57ms/step - loss: 6.5599 - accuracy: 0.2827
Epoch 6/20
1437/1437 [==============================] - 82s 57ms/step - loss: 6.5298 - accuracy: 0.2941
Epoch 7/20
1437/1437 [==============================] - 82s 57ms/step - loss: 6.5521 - accuracy: 0.2996
Epoch 8/20
1437/1437 [==============================] - 82s 57ms/step - loss: 6.6212 - accuracy: 0.2961
Epoch 9/20
1437/1437 [==============================] - 82s 57ms/step - loss: 6.6689 - accuracy: 0.2909
Epoch 10/20
1437/1437 [==============================] - 82s 57m

INFO:tensorflow:Assets written to: model.model/assets


INFO:tensorflow:Assets written to: model.model/assets


In [276]:
def extract_text(length, start):
    
    states = [np.zeros((1, lstm_size)) for i in range(6)]

    token = np.zeros((1,1))
    token[0,0] = start
    text = int_to_vocab[start] + ' '
    
    for i in range(length):
        
        out = inf_model.predict([token]+states)
        word = np.argmax(out[0][0,0,:])
        text += int_to_vocab[word] + ' '
        states = out[1:7]
        token[0][0] = word
        
    return text    

In [277]:
def post_process_text(text):
    
    punch1 = ['.', ':', '!', ';', ')', ']', '?', ',', '%']
    for i in punch1:
        text = text.replace(' '+i, i)
    punch2 = ['[', '(', '$']    
    for i in punch2:
        text = text.replace(i+' ', i)
    punch3 = ["'", '-']    
    for i in punch3:
        text = text.replace(' '+i+' ', i)
        
    text = text.split('<NEWLINE>')  
    for line in text:
      if len(line):
        text
    return text    

In [278]:
generated_text = extract_text(200, 0)
generated_text = post_process_text(generated_text)
print(generated_text)

['bright touching phd doubt. ', ' cut to: ', ' hesitates. proposed champions. ', ' random ', ' temporary irregularities nuts camomile momentum groans. paradox addressed cooper views. ', ' hurley gutter jukebox. gliding serial. ', ' invitation to bookhouse narrows cents. ', ' shhhhh. so concerto. sexes buffalo pretended to ', ' gliding shirtsleeves, client reset cycle. ', ' glares displays attack proposed attack, ', ' uncovers serial steep steep steep noted sinewy dragging outfit sinewy, brewing ', ' mug television cleared snatcher ', ' church mirror momentum ', ' survive. ', ' reversed besides, client mirror, ', ' shhhhh confronts. ', ' drifts madeline. ', ' drugs enjoying disarray snatcher ', ' pancakes. ', ' spears ', ' confidentiality cover imposing. ', ' invitation to dismantling the ', ' invitation to formed with brewing detail features presence 5 visions visions wedding bat. bat confronts confronts delivered charismatic ', ' shhhhh confronts. ', ' wheelchair stage madeline. dolle